In [1]:
import os
import datetime
import numpy as np
import pandas as pd
from os.path import dirname, abspath
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from data_processor import series_to_supervised,evaluate
from data_processor import series_to_supervised,evaluate
from sklearn.svm import SVR

In [2]:
from keras import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [5]:
names = locals()

## modules

### sample generator

In [6]:
def sample_gen(power_train, ):
    # normalizaion
    index = power.index
    scaler = MinMaxScaler(feature_range=(0, 1))
    power_normalized = pd.DataFrame(scaler.fit_transform(power))
    power_normalized['index'] = index
    power_normalized = power_normalized.set_index('index')

    power_supervised = series_to_supervised(power_normalized, 48, 16)

    index_of_inputs = list(range(16)) + list(range(-1, -9, -1))
    inputs = power_supervised.iloc[:, :48]
    inputs = inputs.iloc[:,index_of_inputs]
    outputs = power_supervised.iloc[:, 48:]
    
    data_set = []
    for i in range(16):
        names['output_%s' % str(i + 1)] = outputs.iloc[:, i]
        output_set.append(names['output_%s' % str(i + 1)])
        
    return inputs, output_set, scaler

### ANN module

In [7]:
def ANN_module(train_input, train_output):
    # parameters
    
    input_len = train_input.shape[1]
    output_step = 1
    batch_size = 512
    epochs = 100
    
    for i in range(16):
        # design network
        names['model_%s'%str(i)] = Sequential()

        names['model_%s'%str(i)].add(Dense(int(input_len/2), input_dim=input_len, activation='relu'))
        names['model_%s'%str(i)].add(Dropout(0.2))

        names['model_%s'%str(i)].add(Dense(output_step, activation='relu'))

        adam = Adam(lr=0.001)

        names['model_%s'%str(i)].compile(loss='mse', optimizer='adam')

        history = names['model_%s'%str(i)].fit(train_input, train_output[i],
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_split=0.1,
                            callbacks=[
                                TensorBoard(log_dir='/tmp/tensorboard', write_graph=True),
                                EarlyStopping(monitor='val_loss', patience=5, mode='auto')
                            ]
                            )
        
        return names['model_%s'%str(i)]

### test module

In [ ]:
def test_module()